In [1]:
!nvidia-smi

Thu Aug 15 00:04:52 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 8.2 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Found existing installation: transformers 4.42.3
Uninstalling transformers-4.42.3:
  Successfully uninstalled transformers-4.42.3
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.9 MB/s eta 0:00:00:00:010:01
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

2024-08-15 00:06:40.204863: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 00:06:40.205012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 00:06:40.330214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_bart = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset

dataset_dialogsum = load_dataset("knkarthick/dialogsum")
dataset_dialogsum

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [8]:
split_lengths = [len(dataset_dialogsum[split])for split in dataset_dialogsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_dialogsum['train'].column_names}")
print("\nDialogue:")

print(dataset_dialogsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_dialogsum["test"][1]["summary"])

Split lengths: [12460, 500, 1500]
Features: ['id', 'dialogue', 'summary', 'topic']

Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Ins

In [9]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [10]:
dataset_dialogsum_pt = dataset_dialogsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [11]:
dataset_dialogsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12460
})

In [12]:
# Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bart)

In [13]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='/kaggle/working/bart-dialogsum',   # Đường dẫn lưu trữ output
    num_train_epochs=3,                            # Số epoch, 3 là khởi đầu tốt
    warmup_steps=500,                              # Giảm số warmup steps để tiết kiệm bộ nhớ
    per_device_train_batch_size=2,                 # Giảm batch size xuống 2 để tránh quá tải bộ nhớ
    per_device_eval_batch_size=2,                  # Giảm batch size tương tự cho evaluation
    weight_decay=0.01,                             # Regularization để tránh overfitting
    logging_steps=50,                              # Tăng logging steps để có nhiều thông tin hơn khi training
    eval_strategy='steps',                         # Đánh giá mô hình sau mỗi vài steps
    eval_steps=1000,                               # Đánh giá sau mỗi 1000 steps
    save_steps=1000,                               # Lưu checkpoint sau mỗi 1000 steps
    gradient_accumulation_steps=8,                 # Tăng số bước tích lũy gradient để mô phỏng batch size lớn hơn
    learning_rate=5e-5,                            # Learning rate phổ biến cho BART
    fp16=True,                                     # Kích hoạt mixed precision để tiết kiệm bộ nhớ
    save_total_limit=3                             # Giới hạn số lượng checkpoint được lưu
)

In [14]:
trainer = Trainer(model=model_bart, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_dialogsum_pt["train"],
                  eval_dataset=dataset_dialogsum_pt["validation"])

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
1000,0.848800,1.095185
2000,0.575200,1.153857


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=2334, training_loss=0.8536778011174925, metrics={'train_runtime': 3694.5824, 'train_samples_per_second': 10.118, 'train_steps_per_second': 0.632, 'total_flos': 2.098023063768269e+16, 'train_loss': 0.8536778011174925, 'epoch': 2.997110754414125})

In [19]:
from transformers import GenerationConfig

# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    
    # Tạo đối tượng GenerationConfig
    generation_config = GenerationConfig(
        max_length=150,
        min_length=30,
        early_stopping=True,
        num_beams=4,
        length_penalty=1.0,
        no_repeat_ngram_size=3,
    )
    
    # Tạo các batch từ dữ liệu
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        # Sử dụng GenerationConfig khi sinh văn bản
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device), 
                                   generation_config=generation_config)
        
        # Giải mã và xử lý hậu kỳ
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                             clean_up_tokenization_spaces=True) 
                             for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    # Tính toán và trả về các chỉ số ROUGE
    score = metric.compute()
    return score

In [21]:
!pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [22]:
from evaluate import load

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load("rouge")

In [24]:
# Tính toán score
score = calculate_metric_on_test_ds(
    dataset_dialogsum['test'][0:10], rouge_metric, trainer.model, tokenizer, 
    batch_size=2, column_text='dialogue', column_summary='summary'
)

# Kiểm tra cấu trúc của score để xác định cách truy cập đúng
print(score)

# Giả sử score[rn] đã là giá trị fmeasure trực tiếp
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

# Tạo DataFrame từ kết quả
import pandas as pd
pd.DataFrame(rouge_dict, index=[f'bart'])

100%|██████████| 5/5 [00:07<00:00,  1.58s/it]


{'rouge1': 0.0043410677187272935, 'rouge2': 0.0, 'rougeL': 0.004376886903482648, 'rougeLsum': 0.00427556978088893}


,rouge1,rouge2,rougeL,rougeLsum
bart,0.004341,0.0,0.004377,0.004276


In [28]:
model_bart.save_pretrained("./bart-dialogsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [29]:
## Save tokenizer
tokenizer.save_pretrained("./tokenizer")

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.json',
 './tokenizer/merges.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [30]:
#Load
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

In [31]:
# Prediction
gen_kwargs = {
    "length_penalty": 1.0, 
    "num_beams": 4, 
    "max_length": 150, 
    "min_length": 30, 
    "early_stopping": True
}

sample_text = dataset_dialogsum["test"][0]["dialogue"]

reference = dataset_dialogsum["test"][0]["summary"]

pipe = pipeline("summarization", model="bart-dialogsum-model", tokenizer=tokenizer)

print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with th